In [10]:
%load_ext blackcellmagic
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
import glob
import yaml
from datetime import date
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv
import ipywidgets as widget
from IPython.display import display
import imageio
 # pyright: reportMissingModuleSource=false 

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [11]:
METADATA = {
    "author": "Magnus Wood",
}

# define notebook parameters
params = {
    "project_stub": "Potato_Fertilizer_Othello",
    "image_format": "*.tif",
    "data_acquisition_date": "Jun22_2020",
    "NDVI_threshold": 0.3,
    "data_import_path": Path.cwd() / "data" / "raw" / "Jun22_2020",
    "data_export_path": Path.cwd() / "data" / "processed" / "Jun22_2020",
    "plot_export_path": Path.cwd() / "image_export" ,
    "ground_truth_path": Path.cwd() / "data" / "raw" / "ground_truth.csv",
}

# export the params file for reference
with open(
    Path.cwd()
    / "config"
    / (str(date.today()) + "_" + params["data_acquisition_date"] + "_" + "params.yaml"),
    "w",
) as file:
    documents = yaml.dump(params, file)

# define camera wavelengths and file image labels in a dict
# RedEdge-MX Dual Camera Imaging System bands
# channel names: blue-444, blue, green-531, green, red-650, red, red-edge-705, red-edge, red-edge-740, nir
rededge_mx_band_wl = {
    "blue-444": 444,
    "blue": 475,
    "green-531": 531,
    "green": 560,
    "red-650": 650,
    "red": 668,
    "red-edge-705": 705,
    "red-edge": 717,
    "red-edge-740": 740,
    "nir": 842,
}

ground_truth = pd.read_csv(params["ground_truth_path"])


In [12]:
### useful functions
def show_image(image):
    """ plot array as img """
    plt.figure(figsize=(8, 30))
    plt.imshow(image, cmap="viridis")

def calc_spec_idx(band_a, band_b) -> np.array:
    """ calculates spectral index from np.array in form:
    (band_a - band_b) / (band_a + band_b), avoids divide by zero error """
    numer = np.subtract(band_a, band_b)
    denom = np.add(band_a, band_b)
    return np.divide(numer, denom, out=np.zeros_like(numer), where=(denom != 0))

# Load numpy array with all the bands
Verify that the loaded image has the expected number of bands before continuing. The order of a numpy array is (bands, y, x)